# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откроим файл с данными и изучим общую информацию

In [ ]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [18]:
data = pd.read_csv('/datasets/data.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


**Вывод**
Столбцы days_employed и total_income имеют пропуски, причем в одинаковом количестве.

In [19]:
print(data.describe())

           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt  total_income  
count      21525.000000  21525.000000  1.935100e+04  
mean           0.972544      0.080883  1.674223e+05  
std            1.420324      0.272661  1.029716e+05  
min            0.000000      0.000000  2.066726e+04  
25%            0.000000      0.000000  1.030532e+05  
50%            0.000000      0.000000  1.450179e+05  
75%            1.000000    

## Предобработка данных

### Обработка пропусков

Пропуски по зарплате заполнены на основании среднего по профессии.
Пропуски в общем трудовом стаже решил заполнить нулями, потому что эта колонка не участвует в дальнейшем анализе.

In [20]:
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x:x.fillna(x.mean()))
data['days_employed'] = data['days_employed'].fillna(0)

### Замена типа данных

In [21]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

**Вывод**
Заменен вещественный тип данных на целочисленный методом astype()

### Обработка дубликатов

In [22]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['children'] = data['children'].abs()
data['days_employed'] = data['days_employed'].abs()
print(data.duplicated().sum()) 

71


**Вывод** В количественных столбцах children и days_employed убраны минусы методом abs(). Текст изменен на строчный в столбцах education и family_status методом lower()

Обнаружены дубликаты строк, удаляем их методом drop_duplicates()

In [23]:
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum()) 

0


### Лемматизация

Определение характерных лемм в форулировках целей кредитования

In [24]:
m = Mystem()

purpose =''
for row in data['purpose']:
    purpose += str(row)
    purpose += " "
lemmas = m.lemmatize(purpose)

print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, ' \n': 1})


Добавляем новый столбец purpose_type с унификацией целей кредитования

In [25]:

def get_purpose_type(data_purpose):
    purpose_lemma = m.lemmatize(data_purpose)
    if  ('недвижимость' in purpose_lemma) or ('жилье' in purpose_lemma):
        return  'недвижимость'
    if  'автомобиль' in purpose_lemma:
        return  'автомобиль'
    if  'образование' in purpose_lemma:
        return  'образование'
    if  'свадьба' in purpose_lemma:
        return  'свадьба'
    return 'прочее'
data['purpose_type'] = data['purpose'].apply(get_purpose_type)
print(data[['purpose','purpose_type']].head(10))
    

                      purpose  purpose_type
0               покупка жилья  недвижимость
1     приобретение автомобиля    автомобиль
2               покупка жилья  недвижимость
3  дополнительное образование   образование
4             сыграть свадьбу       свадьба
5               покупка жилья  недвижимость
6           операции с жильем  недвижимость
7                 образование   образование
8       на проведение свадьбы       свадьба
9     покупка жилья для семьи  недвижимость


Проверяем что у нас осталось в "прочих" целях 

In [26]:
print(data.loc[data['purpose_type'] == 'прочее'])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, purpose_type]
Index: []


**Вывод**
Создана колонка 'purpose_type' с классификацией целей кредитования по следующими видами:
 'недвижимость'
 'автомобиль'
 'образование'
 'свадьба'

### Категоризация данных

Создаем словарь education_dict

In [27]:
education_dict = data[['education','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
print(education_dict)

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


Создаем словарь family_status_dict

In [28]:
family_status_dict = data[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop = True)
print(family_status_dict)

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  не женат / не замужем                 4


**Вывод**
Созданы словари для колонок family_status и education

## Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
debt_with_children =  data.loc[data['children']>0]['debt'].sum() / data.loc[data['children']>0]['debt'].count()
print("Доля просроченных кредитов среди заемщиков имеющих детей: {:.1%}".format(debt_with_children))
debt_without_children =  data.loc[data['children']==0]['debt'].sum() / data.loc[data['children']==0]['debt'].count()
print("Доля просроченных кредитов среди заемщиков не имеющих детей: {:.1%}".format(debt_without_children))
print("Кол-во заемщиков имеющих детей:",data.loc[data['children']>0]['children'].count())
print("Кол-во заемщиков не имеющих детей:",data.loc[data['children']==0]['children'].count())

Доля просроченных кредитов среди заемщиков имеющих детей: 9.2%
Доля просроченных кредитов среди заемщиков не имеющих детей: 7.5%
Кол-во заемщиков имеющих детей: 7363
Кол-во заемщиков не имеющих детей: 14091


**Вывод**
Доля просроченных кредитов среди заемщиков не имеющих детей на 1,7% ниже, чем у заемщиков имеющих детей. Зависимость имеется.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
data_family_status = data.groupby(['family_status']).agg({'family_status':'count','debt':'sum'})
data_family_status = data_family_status.rename(columns={'family_status': 'count'})
data_family_status['dept_percent'] = data_family_status['debt']/data_family_status['count']
print(data_family_status.sort_values(by = 'dept_percent', ascending = False))


                       count  debt  dept_percent
family_status                                   
не женат / не замужем   2810   274      0.097509
гражданский брак        4151   388      0.093471
женат / замужем        12339   931      0.075452
в разводе               1195    85      0.071130
вдовец / вдова           959    63      0.065693


**Вывод**
Зависимость есть. Чаще всего просрочку по кредиту допускают заемщики с семейным статусом "не женат / не замужем". Самые дисциплинированные заемщики с семейным статусом "вдовец / вдова".

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разделяем уровень дохода на категории

In [31]:
def get_total_income_type (total_income):
    if total_income < 100000:
        return 'менее 100 тыс.'
    if total_income < 150000:
        return 'от 100 тыс. до 150 тыс.'
    if total_income < 200000:
        return 'от 150 тыс. до 200 тыс.'
    return 'более 200 тыс.'
data['total_income_type'] = data['total_income'].apply(get_total_income_type)
print(data[['total_income','total_income_type']].head())

   total_income        total_income_type
0        253875           более 200 тыс.
1        112080  от 100 тыс. до 150 тыс.
2        145885  от 100 тыс. до 150 тыс.
3        267628           более 200 тыс.
4        158616  от 150 тыс. до 200 тыс.


In [32]:
data_total_income = data.groupby(['total_income_type']).agg({'total_income':'count','debt':'sum'})
data_total_income = data_total_income.rename(columns={'total_income': 'count'})
data_total_income['dept_percent'] = data_total_income['debt']/data_total_income['count']
print(data_total_income.sort_values(by = 'dept_percent', ascending = False))

                         count  debt  dept_percent
total_income_type                                 
от 150 тыс. до 200 тыс.   5333   473      0.088693
от 100 тыс. до 150 тыс.   6090   526      0.086371
менее 100 тыс.            4463   354      0.079319
более 200 тыс.            5568   388      0.069684


**Вывод**
Зависимость присутствует. Просрочку по кредитам чаще допускают заемщики со средним достатком (от 100 тыс. до 200 тыс.). Наиболее ответственно к кредитам относятся заещики с доходом более 200 тыс.

In [33]:
print(data['total_income'].quantile([.25, .5,.75]))

0.25    107623.0
0.50    151887.0
0.75    202417.0
Name: total_income, dtype: float64


- Как разные цели кредита влияют на его возврат в срок?

In [34]:
data_purpose = data.groupby(['purpose_type']).agg({'purpose_type':'count','debt':'sum'})
data_purpose = data_purpose.rename(columns={'purpose_type': 'count'})
data_purpose['dept_percent'] = data_purpose['debt']/data_purpose['count']
print(data_purpose.sort_values(by = 'dept_percent', ascending = False))

              count  debt  dept_percent
purpose_type                           
автомобиль     4306   403      0.093590
образование    4013   370      0.092200
свадьба        2324   186      0.080034
недвижимость  10811   782      0.072334


**Вывод**
Зависимость присутствует. Просрочку по кредиту чаще допускают заемщики, которые его взяли на автомобиль или образование.

## Общий вывод

На риск невозврата в срок кредита влияют:
- наличие детей (выще риск у имеющих детей)
- семейное положение (выше риск у заемщиков со статусом "не женат / не замужем")
- уровень дохода (выше риск у заемщиков со среднем доходом 100-200 тыс)
- цель кредита (выше риск у заемщиков, которые берут кредит на покупку автомобиля или образование)